In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display, Image, Math
from scipy.integrate import solve_ivp
from scipy.signal import place_poles
import control.matlab as control

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

## Lineares LTI System 2. Ordnung

\begin{align*}
    \dot{\boldsymbol{x}}(t) &= \begin{pmatrix}
		0 & 1 \\ -3 & -4
	\end{pmatrix} \, \boldsymbol{x}(t) + \begin{pmatrix}
	0 \\ 3 
\end{pmatrix} \, \boldsymbol{u}(t) \\
     y(t) &= \begin{pmatrix} 1 & 0 \end{pmatrix} \, \boldsymbol{x}(t)
\end{align*}

## Simulationsparameter

In [ ]:
tSim = np.linspace(0, 3, 301)
x0 = [2, 0]
A = np.array([[0, 1], [-3, -4]])
b = np.array([[0], [3]])
c = np.array([[1], [0]])

## Lineares System mit Zustandsregler ohne Vorfaktor

In [ ]:
def linSystemStateCont(t, x, A, b, k):
    
    u = -k @ x

    dx = A @ x + b @ u

    return dx

## Analyse Eigenwerte

In [ ]:
eigA = np.linalg.eig(A)[0]
Math(r's_1 = ' + str(eigA[0]) + r',~s_2 = ' + str(eigA[1]))

## Überprüfen Steuerbarkeit

In [ ]:
S = control.ctrb(A, b)
Math(r'\mathcal{S} = ' + sp.latex(sp.Matrix(S)))

In [ ]:
rank = np.linalg.matrix_rank(S)
Math(r'\mathrm{rang}(\mathcal{S}) = ' + str(rank)) 

## Reglerentwurf

In [ ]:
Q = np.diag([10, 1])
R = np.array([1])

In [ ]:
k, P, EigCL = control.lqr(A, b, Q, R)
Math(r'\boldsymbol{k}=' + sp.latex(sp.Matrix(k)) + ' \quad ' 
      + r'\boldsymbol{P}='+ sp.latex(sp.Matrix(P)) 
      + r'\boldsymbol{\lambda}=' + sp.latex(sp.Matrix(EigCL)))

## Simulation

In [ ]:
k = np.array(k)

In [ ]:
res = solve_ivp(linSystemStateCont, [tSim[0], tSim[-1]], x0,
                       t_eval=tSim, args=(A, b, k))

In [ ]:
plt.figure()
plt.plot(res.t, res.y[0, :])
plt.plot(res.t, res.y[1, :])
plt.grid()
plt.xlabel('t [s]')
plt.ylabel('y(t)')

## Lineares System mit Zustandsregler mit Vorfilter

In [ ]:
def linSystemStateContFactor(t, x, yRef, A, b, c, k):
    
    V = 1 / (c.T @ np.linalg.inv(b @ k - A) @ b)
    
    u = V[0] * yRef - k @ x

    dx = A @ x + b @ u

    return dx

In [ ]:
output = widgets.Output()

with output:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [1, 1, 1]})

plt.subplots_adjust(wspace=0.2, hspace=0.07)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([0, tSim[-1]]) 
ax2.set_xlim([0, tSim[-1]])
ax3.set_xlim([0, tSim[-1]]) 
ax1.set_ylim([-0.05, 3.55]) 
ax2.set_ylim([-2, 2]) 
ax3.set_ylim([-2, 2]) 
ax1.grid()
ax2.grid()
ax3.grid()
ax1.set_ylabel(r"$y$")
ax2.set_ylabel(r"$e$")
ax3.set_ylabel(r"$u$")
ax3.set_xlabel(r"$t$")

ySys, = ax1.plot([], [], 'C0', label=r'Prozess $y = x_1$')
x2Sys, = ax1.plot([], [], 'C3', label=r'Zustand 2 $x_2$')
yRefSys, = ax1.plot([], [], 'C1', label=r'Referenz $y_\mathrm{r}$')
eSys, = ax2.plot([], [], 'C0', label=r'Fehler $e$')
uFBSys, = ax3.plot([], [], 'C2', label=r'Regler $u$')

handlesAx, labelsAx = ax1.get_legend_handles_labels()

fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.13, 0.94, 0.7675, .15), loc=3,
           ncol=3, mode="expand", borderaxespad=0., framealpha=0.5)

sliderYr = widgets.FloatSlider(value=2,
                               min=0,
                               max=5,
                               step=0.5,
                               description=r'$y_{\mathrm{r}}$')
sliderQ1 = widgets.FloatSlider(value=10,
                               min=0.01,
                               max=1000,
                               step=1,
                               description=r'$q_{11}$')
sliderQ2 = widgets.FloatSlider(value=1,
                               min=0.01,
                               max=1000,
                               step=1,
                               description=r'$q_{22}$')
sliderR = widgets.FloatSlider(value=1,
                               min=0.01,
                               max=1000,
                               step=1,
                               description=r'$R$')
sliderX01 = widgets.FloatSlider(value=0,
                               min=-10,
                               max=10,
                               step=0.1,
                               description=r'$x_1(0)$')
sliderX02 = widgets.FloatSlider(value=0,
                               min=-10,
                               max=10,
                               step=0.1,
                               description=r'$x_2(0)$')

def calcSystem(_):
    yr = sliderYr.value
    
    Q = np.diag([sliderQ1.value, sliderQ2.value])
    R = np.array([sliderR.value])
    k, _, _  = control.lqr(A, b, Q, R)
    k = np.array(k)
    
    x0 = [sliderX01.value, sliderX02.value]
    res = solve_ivp(linSystemStateContFactor,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim, args=(yr, A, b, c, k))
    
    V = 1 / (c.T @ np.linalg.inv(b @ k - A) @ b)
           
    sigFB = [V * yr - k @ res.y.T[idx + 1, :] for idx, t in enumerate(tSim[1:])]
    sigE = yr - res.y.T[:, 0]
    uFBSys.set_data(tSim[1:], sigFB)
    ySys.set_data(tSim, res.y.T[:, 0])
    x2Sys.set_data(tSim, res.y.T[:, 1])
    yRefSys.set_data(tSim[[0, -1]], np.array([yr, yr]))
    eSys.set_data(tSim, sigE)
    
    uMax = np.max(sigFB)
    ax1.set_ylim(np.min(res.y) - 0.5, np.max(res.y) + 0.5)
    ax2.set_ylim([(np.min(sigE) - 0.2), (np.max(sigE) + 0.2)]) 
    ax3.set_ylim([-(uMax + 1), (uMax + 1)]) 
    
    fig.canvas.draw()

sliderYr.observe(calcSystem, names='value')
sliderQ1.observe(calcSystem, names='value')
sliderQ2.observe(calcSystem, names='value')
sliderR.observe(calcSystem, names='value')
sliderX01.observe(calcSystem, names='value')
sliderX02.observe(calcSystem, names='value')

calcSystem(_)

stateContControls = VBox([sliderYr, sliderQ1, sliderQ2, sliderR])
stateContControls.layout = make_box_layout()
sysControls = VBox([sliderX01, sliderX02])
sysControls.layout = make_box_layout()

controls = HBox([sysControls, stateContControls])
VBox([controls, output], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))